In [3]:
import urllib.request
import urllib.parse

import scrapy
import requests
from scrapy.http import TextResponse

In [1]:
# !scrapy startproject justwatch
# !rm -rf justwatch

In [6]:
%%writefile justwatch/justwatch/selenium_test.py


from selenium import webdriver
from selenium.webdriver.chrome.options import Options

CHROMEDRIVER_PATH = '0.Scrap\justwatch\justwatch\drivers\chromedriver.exe'
WINDOW_SIZE = "1920,1080"
 
chrome_options = Options()
# chrome_options.add_argument( "--headless" )     # 크롬창이 열리지 않음
# chrome_options.add_argument( "--no-sandbox" )   # GUI를 사용할 수 없는 환경에서 설정, linux, docker 등
# chrome_options.add_argument( "--disable-gpu" )  # GUI를 사용할 수 없는 환경에서 설정, linux, docker 등
# chrome_options.add_argument(f"--window-size={ WINDOW_SIZE }")  # 브라우저가 설정한 창크기로 실행됩니다.
chrome_options.add_argument('--start-maximized')  # 브라우저가 최대화된 상태로 실행됩니다.
chrome_options.add_argument('Content-Type=application/json; charset=utf-8')
 
driver = webdriver.Chrome( executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options )
driver.get( 'https://news.naver.com' )

Overwriting justwatch/justwatch/selenium_test.py


In [10]:
%%writefile justwatch/justwatch/items.py
import scrapy

class JustwatchItem(scrapy.Item):
    title = scrapy.Field()

Overwriting justwatch/justwatch/items.py


In [1]:
# %%writefile justwatch/justwatch/spiders/spider.py

# aw

Writing justwatch/justwatch/spiders/spider.py


In [4]:
from fake_useragent import UserAgent

url = 'https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from=2022'
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8") 
response

<200 https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from=2022>

In [ ]:
response.text

In [8]:
links = response.xpath('//*[@id="base"]/div[3]/div/div[2]/div[1]/div/div/a/@href').extract()
links = list(map(response.urljoin, links))
links
# print(response.text)

['https://www.justwatch.com/kr/영화/das-privileg-die-auserwahlten',
 'https://www.justwatch.com/kr/영화/restless-2022',
 'https://www.justwatch.com/kr/영화/neoneun-nayi-ufo',
 'https://www.justwatch.com/kr/영화/tegsaseu-jeongitob-hagsal-2022',
 'https://www.justwatch.com/kr/영화/behind-the-scenes-with-jane-campion',
 'https://www.justwatch.com/kr/영화/eoleobeorin-sigan-sogeseo',
 'https://www.justwatch.com/kr/영화/moreolsenseu',
 'https://www.justwatch.com/kr/영화/ai-love-you',
 'https://www.justwatch.com/kr/영화/the-weekend-away',
 'https://www.justwatch.com/kr/영화/tyler-perrys-a-madea-homecoming',
 'https://www.justwatch.com/kr/영화/bigbeogeu',
 'https://www.justwatch.com/kr/영화/boiji-anhneun-ggeun',
 'https://www.justwatch.com/kr/영화/fistful-of-vengeance',
 'https://www.justwatch.com/kr/영화/downfall-the-case-against-boeing',
 'https://www.justwatch.com/kr/영화/heart-shot',
 'https://www.justwatch.com/kr/영화/haejeog-doggaebi-gisbal',
 'https://www.justwatch.com/kr/영화/munich-the-edge-of-war',
 'https://www.just

In [35]:
url = 'https://www.justwatch.com/kr/영화/das-privileg-die-auserwahlten'
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8") 
response

<200 https://www.justwatch.com/kr/%EC%98%81%ED%99%94/das-privileg-die-auserwahlten>

In [126]:

title = response.css("div.title-block > div > h1::text").get().strip()
opening_date = response.css("div.title-block > div > span::text").get().replace("(","").replace(")","").strip()
rating = response.css("div.title-info visible-xs visible-sm >div.jw-scoring-listing__rating > a::text").get()
just_rating = response.css('div.detail-infos > div > div > div > a::text')[0].get().strip()
imdb_rating = response.css('div.detail-infos > div > div > div > a::text')[1].get().strip()
genre = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div/div[2]/span/text()').getall()     # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
genre = ",".join(genre).replace(" ","")
runtime = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[2]/text()').getall()
runtime = ",".join(runtime).replace(" ","")
director = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[4]/div[2]/span/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
director = ",".join(director).replace(" ","")
actors = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[3]/div/div/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음...
actors = ",".join(actors).replace(" ","")
synopsis = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[4]/p/span/text()').get()




# try:
#     item['synopsis'] = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[4]/p/span/text()').get()
# except:
#     item['synopsis']=None
#print("제목:",title_kor, "개봉일:",opening_date,"저스트와치 평점:", just_rating,"IMDB 평점:",imdb_rating, "장르:",genre, "재생시간:",runtime, "감독:",director,"출연자:", actors, "시놉시스:",synopsis)
# # print(actors_list)    # 함수값 안에서의 출연진 값이 print 값으로 표시됨
# yield item

In [125]:
print(title, opening_date, rating, just_rating, imdb_rating, genre, runtime, director, actors, sep=',')

더 프리빌리지,2022,None,59%,4.6 (3k),드라마,공포,SF,스릴러,1시간47분,FelixFuchssteiner,KatharinaSchöde,MaxSchimmelpfennig,LeavanAcken,TijanMarei,MilenaTscharntke,LiseRisomOlsen,RomanKnizka,NadeshdaBrennicke,HorstJanson,CarolineHartig,JeanetteSpassova,OlegTikhomirov,MauriceLattke,RojanJuanBarani


In [128]:
type(synopsis)

str

In [119]:
# just_rating.strip()
# opening_date.strip()
imdb_rating.strip()

'4.6 (3k)'

In [54]:
c = response.css("div.title-block > div > h1::text").get().strip()
c

'더 프리빌리지'

In [130]:
import pandas as pd

In [131]:
list_data = ['2019-01-02',3.14,'ABC',100,True]
series_data = pd.Series(list_data)

## test css

In [57]:
txt = response
print(txt)

<200 https://www.justwatch.com/kr/%EC%98%81%ED%99%94/das-privileg-die-auserwahlten>


In [ ]:
response.text

In [106]:
# 장르,감독 배우


genre = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div/div[2]/span/text()').getall()     # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
genre = ",".join(genre).replace(" ","")
runtime = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[2]/text()').getall()
runtime = ",".join(runtime).replace(" ","")
director = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[4]/div[2]/span/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
director = ",".join(director).replace(" ","")
actors = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[3]/div/div/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음...
actors = ",".join(actors).replace(" ","")

In [107]:
genre, runtime, director, actors

('드라마,공포,SF,스릴러',
 '1시간47분',
 'FelixFuchssteiner,KatharinaSchöde',
 'MaxSchimmelpfennig,LeavanAcken,TijanMarei,MilenaTscharntke,LiseRisomOlsen,RomanKnizka,NadeshdaBrennicke,HorstJanson,CarolineHartig,JeanetteSpassova,OlegTikhomirov,MauriceLattke,RojanJuanBarani')

In [88]:
response.xpath('//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[2]/div[2]/span/text()').extract_first()

'드라마 '

In [98]:
a = response.xpath('//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[2]/div[2]/span/text()').extract()
a = ",".join(a).replace(" ","")

In [99]:
a

'드라마,공포,SF,스릴러'

In [100]:
type(a)

str